In [1]:
import numpy as np 

In [2]:
from sia import Pipeline
from sia.io import Metadata, read_edf, read_csv, read_dataset, write_csv, write_png, write_dataset
from sia.preprocessors import neurokit
from sia.transformers import as_window, scaleogram

In [3]:
def encode_category(target):
    def inner(label):
        return {
            'label': np.isin(label, target).astype(int)
        }
    return inner

In [4]:
def resize(width: int, height: int):
    def inner(image):
        return {
            'pixel_values': [x.convert("RGB").resize((width, height)) for x in image]
        }
    return inner

In [ ]:
Pipeline() \
    .data(
        read_edf(
            './data/ecg_raw/*.edf', 
            Metadata('./data/ecg_raw/TimeStamps_Merged.txt').on_regex(r'[0-9]{5}')
        )
    ) \
    .process(neurokit()) \
    .to(write_csv('./data/ecg_preprocessed/[0-9]{5}.csv'))

In [ ]:
Pipeline() \
    .data(read_csv('./data/ecg_preprocessed/*.csv', ['ECG_Clean', 'category'])) \
    .rename({'ECG_Clean': 'signal', 'category': 'label'}) \
    .process(encode_category(['TA', 'SSST_Sing_countdown', 'Pasat', 'Raven', 'TA_repeat', 'Pasat_repeat'])) \
    .to(write_csv('./data/ecg_model/[0-9]{5}.csv'))

In [ ]:
# Pipeline() \
#     .data(read_csv('./data/ecg_preprocessed/*.csv', ['ECG_Clean', 'category'])) \
#     .rename({'ECG_Clean': 'signal', 'category': 'label'}) \
#     .process(encode_category(['TA', 'SSST_Sing_countdown', 'Pasat', 'Raven', 'TA_repeat', 'Pasat_repeat'])) \
#     .process(as_window(1000)) \
#     .drop('signal') \
#     .to(write_csv('./data/ecg_windowed_model/[0-9]{5}.csv'))

In [ ]:
Pipeline() \
    .data(read_csv('./data/ecg_preprocessed/*.csv', ['ECG_Clean', 'category'])) \
    .rename({'ECG_Clean': 'signal', 'category': 'label'}) \
    .process(scaleogram('morl')) \
    .to(write_png('./data/scaleograms/[0-9]{5}/'))

In [ ]:
Pipeline() \
    .data(read_dataset('./data/scaleograms/**/')) \
    .process(resize(127, 127)) \
    .drop('image') \
    .to(write_dataset('./data/scaleograms_resized/[0-9]{5}/'))